# import

In [1]:

import pandas as pd
from tqdm.notebook import tqdm as tqdm
import time
from graphviz import *
import lightgbm as lgb
from hyperopt import hp, tpe, Trials, fmin,STATUS_OK
import fasttext as ft
# import importlib
# importlib.reload(hoge)
from my_library.horse import *
import pickle


# path

In [2]:
path_ubu = '/home/hipro/デスクトップ/Horse/Data/20_21'
path_win2 = '/Users/rince/Desktop/Horse/Data/saishin2/'
path_win = '/Users/rince/Desktop/Horse/Data/saishin/'
path_win = '/Users/Owner/Desktop/program/Horse/Data/saishin/'
path_win2 = '/Users/Owner/Desktop/program/Horse/Data/saishin2/'
path_ft = '/Users/Owner/Desktop/Horse/ft_data/peds_ft.txt'

# load data

In [3]:
results = load_csv(path_win+'results.csv')
horse_results = load_csv(path_win+'horse_results.csv')
peds = load_csv(path_win+'peds.csv')
# 何回やってもロードすると, nanが出る
peds.fillna('nan',inplace=True)
return_tables = load_csv(path_win+'return.csv')
return_tables.rename(columns={'0':0,'1':1,'2':2,'3':3},inplace=True)

In [ ]:
len(set(results.index))

In [ ]:
len(set(return_tables.index))

# race_id 命名規則

race_id 202105040802\
yyyy_pp_xx_xxrr\
y : year\
p : palce\
x : 謎\
r : race番号

In [ ]:
ll = LearnLGBM(peds,results,horse_results)
ll.path_ft = '/Users/Owner/Desktop/Horse/horse/peds_ft.txt'
ll.learn_lgb()

# R & D

In [ ]:

ll.learn_model_ft()

In [ ]:
train_query = ll.x_train.groupby(ll.x_train.index).size()
train = lgb.Dataset(ll.x_train, ll.y_train, group=train_query)
ll.learn_lgb2(train,lgbm_params=lgbm_params)


In [ ]:
# ll = LearnLGBM(peds,results,horse_results)
# ll.get_train_data()


val_x = ll.x_test.iloc[:-10000]
val_y = ll.y_test.iloc[:-10000]
test_query = val_x.groupby(val_x.index).size()
train_query = ll.x_train.groupby(ll.x_train.index).size()
train = lgb.Dataset(ll.x_train, ll.y_train, group=train_query)
valid = lgb.Dataset(val_x, val_y, reference=train, group=test_query)

# 正解ラベル　変えた

In [ ]:
def split_data(df, test_size=0.2, rank_learning=True):
    """
    データを学習データと, 訓練データに分ける関数
    """
    df_ = df.copy()
    df_['rank'] = df_['rank'].map(lambda x: int((1/x)*10) if x<4 else(0))
        
    sorted_id_list = df_.sort_values("date").index.unique()
    train_id_list = sorted_id_list[: round(len(sorted_id_list) * (1 - test_size))]
    test_id_list = sorted_id_list[round(len(sorted_id_list) * (1 - test_size)) :]
    train = df_.loc[train_id_list]#.drop(['date'], axis=1)
    test = df_.loc[test_id_list]#.drop(['date'], axis=1)
    return train, test

    
train, test = split_data(pt.r.data_c.fillna(0),test_size=0.2,rank_learning=False)
x_train = train.drop(['rank', 'date','単勝'], axis=1)
y_train = train['rank']
x_test = test.drop(['rank', 'date','単勝'], axis=1)
y_test = test['rank']
train_query = x_train.groupby(x_train.index).size()
train = lgb.Dataset(x_train, y_train, group=train_query)
# val_y = y_test.iloc[:-10000]
test_query = x_test.groupby(x_test.index).size()
valid = lgb.Dataset(x_test, y_test, reference=train, group=test_query)

In [ ]:
lgbm_params = {
                'metric': 'ndcg',
                'objective': 'lambdarank',
                'ndcg_eval_at': [1,2,3],
                'boosting_type': 'gbdt',
                'random_state': 777,
                # 'lambdarank_truncation_level': 37,
                #  'learning_rate': 0.01271180569095827,
                'lambdarank_truncation_level': 10,
                'learning_rate': 0.02273417953255777,
                'n_estimators': 97,
                'num_leaves': 42,
                'force_col_wise':True
            }



In [ ]:
import warnings

warnings.filterwarnings("ignore")
ll.learn_lgb2(train,lgbm_params=lgbm_params)

In [ ]:
def score(params):
    print("Training start:")

    N_boost_round = []
    Score = []

    lgb_results={}  #履歴格納用
    # train = lgb.Dataset(x_train, y_train, group=train_query)
    # valid = lgb.Dataset(x_test, y_test, reference=train, group=test_query)

    
    lgb_clf = lgb.train(
        params,
        train,
        num_boost_round=1000,
        valid_sets=valid,
        valid_names=['valid'],
        early_stopping_rounds=20,
        verbose_eval=5,
        evals_result=lgb_results,
    )
#     return lgb_results
    return {'loss': -1.0 * lgb_results['valid']['ndcg@3'][lgb_clf.best_iteration], 'status': STATUS_OK}

def optimize(trials):
#探索スペース
    space = {
        'objective': 'lambdarank',
        'metric': 'ndcg',
        # 上位三着を考慮する
        'ndcg_eval_at': [1,2,3],
#         best paramsの返り値は, choiceだとindexか?
        'lambdarank_truncation_level': hp.choice('lambdarank_truncation_level',range(1,50)),
        'learning_rate': hp.uniform('learning_rate', 0.01, 0.1),
        # n_estimators == num_iterations
        # 'num_iterations': hp.choice('num_iterations',range(50,120)),
        # 'num_leaves': hp.choice('num_leaves',range(20,50)),
        'boosting_type': 'gbdt',
        'random_state': 777
    }

    max_evals = 25      #探索回数(25くらいで十分)
    best = fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=max_evals)

    print("best parameters:", best)

In [ ]:
trials = Trials()
optimize(trials)

# Simulate


In [4]:
# !pwd
save_path = '/Users/Owner/Desktop/Horse/horse/ll_obj.pickle'

def save_pickle(save_path,object_):
    with open(save_path, mode="wb") as f:
        pickle.dump(object_, f)

def load_pickle(save_path):
    with open(save_path, mode="rb") as f:
        object_ = pickle.load(f)
    return object_

# save_pickle(save_path,ll)
ll = load_pickle(save_path)
ll.learn_model_ft()


In [6]:

for odds in [1.1]:
    print()
    sl = RankSimulater(ll.model)
    print("odds",odds)
    # profit, is_atari, is_buy, actual_rank, not_buy_reason, is_error =  sl.calc_tansho(ll.r.data_c,202109060906,odds,100,True)
    result_df = sl.show_sim_results(ll.r.data_c.loc[ll.x_test.index[0]:],return_tables)



odds 1.1
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5


In [15]:
result_df['profit']

202205020201        0
202205020202     -100
202205020203    250.0
202205020204        0
202205020205     -100
                ...  
202207010808    200.0
202207010809     -100
202207010810     -100
202207010811     -100
202207010812     -100
Name: profit, Length: 1783, dtype: object

In [ ]:
sl = RankSimulater(ll.model)
sl.calc_sanrenpuku(ll.r.data_c.iloc[-5000:].fillna(0),return_tables)

In [ ]:
sl = RankSimulater(ll.model)
sl.calc_sanrentan(ll.r.data_c.iloc[-5000:].fillna(0),return_tables)

# race_id_list

In [5]:

race_id_list = ['2022040302{}'.format(str(i).zfill(2)) for i in range(1,13)]
race_id_list += ['2022100402{}'.format(str(i).zfill(2)) for i in range(1,13)]
race_id_list += ['2022010202{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022050308{}'.format(str(i).zfill(2)) for i in range(1,13)]

# race_id_list += ['2022090301{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022090302{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022090303{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022090304{}'.format(str(i).zfill(2)) for i in range(1,13)]

# race_id_list += ['2022020103{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022020104{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022020105{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022020106{}'.format(str(i).zfill(2)) for i in range(1,13)]



# Predictor

In [9]:
ll.model_ft

In [6]:
pt = Predictor(peds,results,horse_results,race_id_list)
# ********* test_size = 0  : ずっと0にしてなかった...
pt.path_ft = ll.path_ft
pt.learn_lgb(test_size=0)

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/18088 [00:00<?, ?it/s]

  0%|          | 0/18088 [00:00<?, ?it/s]

...

  0%|          | 0/261 [00:00<?, ?it/s]

  0%|          | 0/261 [00:00<?, ?it/s]

  0%|          | 0/261 [00:00<?, ?it/s]

C:\Users\Owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Total Bins 49090
[LightGBM] [Info] Number of data points in the train set: 114014, number of used features: 172


In [11]:
for race_id in race_id_list:
    print(race_id)
    pt.predict(race_id)

202204030201


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

              馬番    scores
202201020212   5 -0.085450
202201020212   3 -0.107679
202201020212   4 -0.287166
202201020212   6 -0.294318
202201020212  12 -0.331234
202201020212   7 -0.379503
202201020212   8 -0.409376
202201020212   9 -0.505118
202201020212   2 -0.533861
202201020212  10 -0.736029
202201020212   1 -0.738335
202201020212  11 -1.740566


In [10]:
pt.r.data_c.loc[int(race_id_list[0])]

KeyError: 202204030201

In [8]:
sl = RankSimulater(pt.model)
result_df = sl.show_sim_results(pt.r.data_c,return_tables)

KeyboardInterrupt: 

In [ ]:
train, test = pt.get_train_data(test_size=0)

In [ ]:
pt.learn_lgb2(train,lgbm_params)

# 東京

In [ ]:
base  = '2022050304'
pt.predict(base+'10')

# 中京

In [ ]:
base  = '2022070404'
# for i in ['07','08','09','10','11']:
pt.predict(base+'10')

# 函館

In [ ]:
base  = '2022020102'
# for i in ['07','08','09','10','11']:
pt.predict(base+'11')

In [ ]:

pt.show_results_today()

# 日付に注意

In [ ]:
date = '2022/12/31'

# Results scraping

In [ ]:
# race_id_list = results.index.astype('str')

results_tmp = Results.scrape(race_id_list)

# Horse_results scraping

In [ ]:
horse_id_list = results_tmp['horse_id'].astype(str).unique()
horse_results_tmp = HorseResults.scrape(horse_id_list)

# save_path = '/Users/rince/Desktop/Horse/Data/horse_2020.csv'

# Peds scraping

In [ ]:
peds_2021 = Peds.scrape(horse_id_list)
pe_2021 = Peds(peds_2021)
pe_2021.regularize_peds()
peds_tmp = pe_2021.peds_re

# Return scraping

In [ ]:
race_id_list = list(map(lambda x:str(x),race_id_list))

In [ ]:
returns_tmp = Return.scrape(race_id_list)


# update

In [ ]:
new_results = update_data(load_csv(path_win+'results.csv'),results_tmp)
new_horse_results = update_data(load_csv(path_win+'horse_results.csv'),horse_results_tmp)
new_peds = update_data(load_csv(path_win+'peds.csv'),peds_tmp)
returns = load_csv(path_win+'return.csv')
returns.rename(columns={'0':0,'1':1,'2':2,'3':3},inplace=True)
returns_tmp.rename(columns={'0':0,'1':1,'2':2,'3':3},inplace=True)
new_return = update_data(returns,returns_tmp)


# save

In [ ]:
new_results.to_csv(path_win+'results.csv')
new_horse_results.to_csv(path_win+'horse_results.csv')
new_peds.to_csv(path_win+'peds.csv')
new_return.to_csv(path_win+'return.csv')

# 重要度

In [ ]:
importances = pd.DataFrame(
{'features' : x_train.columns, 'importances' : lgb_rank.feature_importance()})
print(importances.sort_values('importances', ascending=False)[:30])

# fastText

流れ
1. fasttext用の血統データの学習データを作る (血統の情報のみ, index ヘッダはいらない)
2. fasttext学習
3. 学習モデルを使って, 血統データをベクトル化
4. ベクトル化して r.data_cに concat
5. 学習


教師あり, 教師なしでも生成されるベクトルは等しい

# model_ft 作成

In [ ]:
# 相対パスしかできない
# dim : 出力の次元
# minn : n_gramの最小単位
# maxn : n_gramの最大単位
path_ft = '/Users/rince/Desktop/Horse/code/horse/peds_ft.txt'
# 上書き保存OK
peds.to_csv(path_ft,header=False,index=False,sep=',')
model_ft = ft.train_unsupervised(path_ft,dim=62,minn=2,maxn=14)

model[model.words[1]] と model.get_input_vector(ind=1) は等価

In [ ]:
lis_ = [1,2,3,4,True,4]
lis2_ = [2,3,4,5,False,5]
dic_ = {
}
dic_['a'] = lis_
dic_['b'] = lis2_

df_ = pd.DataFrame(dic_).T

In [ ]:
df_

In [ ]:
df_.rename(columns={0:'profit',1:'is_atari',2:'is_buy',3:'actual_rank',4:'not_buy_reason',5:'is_error'})